In [ ]:
import pandas as pd
import sqlite3


In [ ]:
from IPython.display import clear_output
clear_output(wait=True)


In [ ]:
# Load CSV file into a pandas DataFrame
df = pd.read_csv('data/Telco-Customer-Churn.csv')
df.head()


In [ ]:
# Convert TotalCharges to numeric (some rows are blank)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Drop rows where TotalCharges is NaN
df = df.dropna(subset=['TotalCharges'])

# Optional: Reset index after dropping rows
df.reset_index(drop=True, inplace=True)

df.info()


In [ ]:
# Create SQLite in-memory database connection
conn = sqlite3.connect(':memory:')

# Load DataFrame into a table named 'customers'
df.to_sql('customers', conn, index=False, if_exists='replace')

print("✅ Data loaded into SQLite")


In [ ]:
query = """
SELECT 
    gender, 
    SeniorCitizen, 
    Partner, 
    Dependents, 
    COUNT(*) AS total_customers,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS churn_rate
FROM customers
GROUP BY gender, SeniorCitizen, Partner, Dependents
ORDER BY churn_rate DESC
LIMIT 10;
"""

churn_rates = pd.read_sql_query(query, conn)
churn_rates


In [ ]:
import matplotlib.pyplot as plt

churn_rates.plot(
    x='churn_rate',
    y='total_customers',
    kind='barh',
    title='Churn Rate by Demographic Group',
    figsize=(10, 6)
)
plt.xlabel("Churn Rate (%)")
plt.ylabel("Customer Group")
plt.show()


In [ ]:
query = """
SELECT 
    Contract, 
    PaymentMethod,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS churn_rate
FROM customers
GROUP BY Contract, PaymentMethod
ORDER BY churn_rate DESC
LIMIT 10;
"""

contract_churn = pd.read_sql_query(query, conn)
contract_churn


In [ ]:
query = """
SELECT 
    Churn,
    ROUND(AVG(MonthlyCharges), 2) AS avg_monthly_charges,
    ROUND(AVG(tenure), 2) AS avg_tenure_months,
    COUNT(*) AS customer_count
FROM customers
GROUP BY Churn;
"""

churn_summary = pd.read_sql_query(query, conn)
churn_summary


In [ ]:
import matplotlib.pyplot as plt

churn_summary.plot(
    x='Churn',
    y=['avg_monthly_charges', 'avg_tenure_months'],
    kind='bar',
    figsize=(8, 6),
    title='Average Monthly Charges and Tenure by Churn Status'
)
plt.ylabel('Average Value')
plt.show()
